# Set up

In [36]:
import numpy as np
import cvxpy as cp
import pandas as pd
from footbot.data import utils
import requests
import json
from footbot.optimiser import team_selector
from footbot.data import element_data

# Getting data

In [3]:
client = utils.set_up_bigquery(
    secrets_path='../secrets/service_account.json'
)

current_sql = \
'''
SELECT
  element,
  element_type,
  value,
  team,
  total_points,
  minutes,
  COALESCE(SAFE_DIVIDE(total_points, minutes), 0) AS points_per_minute,
  safe_web_name
FROM (
  SELECT
    element,
    element_type,
    now_cost AS value,
    team,
    total_points,
    minutes,
    safe_web_name,
    ROW_NUMBER() OVER(PARTITION BY element ORDER BY datetime DESC) AS is_most_recent
  FROM
    `footbot-001.fpl.element_data_1920` )
WHERE
  is_most_recent = 1
'''

current_df = client.query(current_sql).to_dataframe()
current_df.head()

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
0,11,4,110,1,44,540,0.081481,aubameyang
1,39,2,44,4,2,67,0.029851,bernardo
2,51,3,54,4,0,0,0.000000,izquierdo
3,61,2,45,3,13,271,0.047970,mepham
4,89,2,40,5,0,0,0.000000,gibson


In [4]:
players = current_df.to_dict('records')

# Optimiser

## From scratch

In [14]:
first_team_elements, captain_elements, bench_elements, transfers = \
team_selector.select_team(
    players,
    total_budget=1000,
    optimise_key='total_points',
    bench_factor=0.1
)
print(first_team_elements)
print(captain_elements)
print(bench_elements)

[215, 304, 191, 463, 210, 460, 297, 447, 219, 278, 388]
[210]
[128, 319, 21, 65]


In [15]:
current_df[current_df['element'].isin(first_team_elements)].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
509,388,1,50,19,31,540,0.057407,fabianski
39,304,2,52,16,28,540,0.051852,vestergaard
329,297,2,44,15,32,508,0.062992,lundstram
368,447,2,47,5,27,540,0.050000,pieters
34,215,3,99,11,54,449,0.120267,de bruyne
89,191,3,125,10,50,528,0.094697,salah
186,463,3,66,6,33,540,0.061111,mount
438,219,3,76,11,36,404,0.089109,david silva
274,210,4,122,11,55,446,0.123318,aguero
314,460,4,75,6,44,403,0.109181,abraham


In [16]:
current_df[current_df['element'].isin(captain_elements)]

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
274,210,4,122,11,55,446,0.123318,aguero


In [17]:
current_df[current_df['element'].isin(bench_elements)].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
331,319,1,45,16,25,540,0.046296,gunn
230,128,2,41,7,16,348,0.045977,kelly
494,65,2,41,3,12,270,0.044444,rico
344,21,3,46,1,19,511,0.037182,guendouzi


## Existing team

### Select team

In [18]:
bootstrap_request = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
bootstrap_data = bootstrap_request.json()

current_event = [i for i in bootstrap_data['events'] if i['is_current']][0]['id']
print(current_event)

6


In [19]:
entry = 695843

In [20]:
entry_request = requests.get(f'https://fantasy.premierleague.com/api/entry/{entry}/event/{current_event}/picks/')
entry_data = entry_request.json()

In [22]:
first_team_elements, captain_elements, bench_elements, transfers = \
team_selector.select_team(
    players,
    total_budget=1000,
    optimise_key='total_points',
    bench_factor=0.01,
    existing_squad_elements=[i['element'] for i in entry_data['picks']],
    transfer_limit=3
)
print(first_team_elements)
print(captain_elements)
print(bench_elements)

[182, 215, 304, 191, 287, 463, 210, 460, 297, 278, 388]
[210]
[48, 126, 271, 65]


In [23]:
current_df[current_df['element'].isin(first_team_elements)].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
509,388,1,50,19,31,540,0.057407,fabianski
32,182,2,71,10,31,540,0.057407,alexander-arnold
39,304,2,52,16,28,540,0.051852,vestergaard
329,297,2,44,15,32,508,0.062992,lundstram
34,215,3,99,11,54,449,0.120267,de bruyne
89,191,3,125,10,50,528,0.094697,salah
129,287,3,50,14,29,526,0.055133,cantwell
186,463,3,66,6,33,540,0.061111,mount
274,210,4,122,11,55,446,0.123318,aguero
314,460,4,75,6,44,403,0.109181,abraham


In [24]:
list(current_df[current_df['element'].isin(first_team_elements)].sort_values('element_type')['safe_web_name'].values)

['fabianski',
 'alexander-arnold',
 'vestergaard',
 'lundstram',
 'de bruyne',
 'salah',
 'cantwell',
 'mount',
 'aguero',
 'abraham',
 'pukki']

In [25]:
current_df[current_df['element'].isin(captain_elements)]

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
274,210,4,122,11,55,446,0.123318,aguero


In [26]:
current_df[current_df['element'].isin(bench_elements)].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
143,48,1,40,4,0,0,0.000000,button
282,126,2,45,7,23,540,0.042593,ward
494,65,2,41,3,12,270,0.044444,rico
287,271,3,45,13,18,540,0.033333,hayden


In [27]:
current_df[current_df['element'].isin(transfers['transfers_in'])].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
509,388,1,50,19,31,540,0.057407,fabianski
32,182,2,71,10,31,540,0.057407,alexander-arnold
34,215,3,99,11,54,449,0.120267,de bruyne


In [28]:
current_df[current_df['element'].isin(transfers['transfers_out'])].sort_values('element_type')

,element,element_type,value,team,total_points,minutes,points_per_minute,safe_web_name
331,319,1,45,16,25,540,0.046296,gunn
165,141,2,61,8,24,522,0.045977,digne
526,214,3,121,11,39,450,0.086667,sterling


### Optimise entry

In [35]:
team_selector.optimise_entry(
    entry,
    total_budget='1000',
    bench_factor='0.1',
    transfer_penalty='0',
    transfer_limit='15'
)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U21'), dtype('<U21')) -> dtype('<U21')

In [39]:
df = element_data.get_element_df()

In [40]:
df['average_points'] = df['total_points'] / df['current_event']

In [44]:
df = df[[
    'element',
    'element_type',
    'now_cost',
    'team',
    'average_points',
    'safe_web_name'
]]

In [45]:
df.columns = [
    'element',
    'element_type',
    'value',
    'team',
    'average_points',
    'safe_web_name'
]